# Testing accuracy of planetary radius predictions

- Useful imports:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, chi2
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

%matplotlib qt 

from planet import Planet
from star import Star
from target_selection import Config

- Useful constants :

In [2]:
MS = 1.989e30  # kg
RS = 6.96342e8  # m
AS = 4.6  # yo
BSsw = 1  # T
LS = 3.826e26  # W

MJ = 1.8986e27  # kg
RJ = 69911e3  # m
wJ = 1.77e-4  # s-1

- Retrieving dataset :

In [3]:
config_param = Config()
data =pd.read_csv(r'exoplanet.eu_catalog.csv')
data = config_param.param_names(data=data)

- Initializing Jupiter and the Sun:

In [4]:
jup = Planet(
    name="Jupiter",
    mass=1.0,
    radius={"models": config_param.planet_radius_models, "radius": 1.0},
    distance=5.2,
    worb={"star_mass": MS, "worb": 1.0},
    luminosity={
        "models": config_param.planet_luminosity_models,
        "luminosity": np.nan,
        "star_age": 4.6,
    },
    wrot=1.0,
)

sun = Star(
    name="Soleil",
    mass=1.0,
    radius={"models": config_param.star_radius_models, "radius": 1.0},
    age=AS,
    obs_dist=1.0,
    sp_type = 'V'
)
print(sun.luminosity)

0.6977165691451518


- Empty lists for computed/not computed radius and with/without radius expansion : 

In [5]:
radius_data = [] ; mass_data = []
radius_estimate = []
radius_estimate_expansion = []
star_distance = []
temp_eq = []

- Computing :

In [6]:
for target in data.itertuples():
    print(target.name)
    if 'PSR' in target.name :
        print('Warning : {} has been skipped'.format(target.name))
        continue

    if (
        not np.isnan(target.semi_major_axis)
        and (not np.isnan(target.mass) or not np.isnan(target.mass_sini))
        and not np.isnan(target.star_mass)
    ):

        if np.isnan(target.mass):
            planet_mass = target.mass_sini * np.sqrt(4 / 3.0)
        else:
            planet_mass = target.mass
        if not np.isnan(target.eccentricity):
            planet_distance = target.semi_major_axis * (1 - target.eccentricity)
        else:
            planet_distance = target.semi_major_axis

        if np.isnan(target.star_age):
            star_age = 5.2
        else:
            if target.star_age <= 1e-7:
                print("Stellar age is too small (< 100 yr) !!!")
                continue
            elif target.star_age < 0.5:
                star_age = 0.5
            else:
                star_age = target.star_age

        target_radius = np.nan
        radius_data.append(target.radius) ; mass_data.append(planet_mass)
        print(target.radius)
        planet = Planet(
            name=target.name,
            mass=planet_mass,
            radius={"models": config_param.planet_radius_models, "radius": target_radius},
            distance=planet_distance,
            worb={"star_mass": target.star_mass, "worb": np.nan},
            luminosity={
                "models": config_param.planet_luminosity_models,
                "luminosity": np.nan,
                "star_age": star_age,
            },
            detection_method=target.detection_type,
            wrot=1.0,
        )
        star = Star(
            name=target.star_name,
            mass=target.star_mass,
            radius={"models": config_param.star_radius_models, "radius": target.star_radius},
            age=star_age,
            obs_dist=target.star_distance,
            sp_type = str(target.star_sp_type)
        )
        star_distance.append(planet.stardist)
        radius_estimate.append(planet.radius)
        print(star.luminosity)
        if np.isnan(target_radius) and config_param.radius_expansion :# and (not np.isnan(target.eccentricity)) :
            Teq = planet.radius_expansion_old(luminosity=star.luminosity, eccentricity=target.eccentricity)
            temp_eq.append(Teq)
            print('here')
            #planet.radius_expansion()
            radius_estimate_expansion.append(planet.radius)

109 Psc b
1.152
1.2562636974548727
here
112 Psc b
nan
1.1048736715071679
here
112 Psc c
nan
1.1048736715071679
here
11 Com b
nan
53.15657000213334
here
11 UMi b
nan
10.213705168807667
here
14 And b
nan
23.499637363842567
here
14 Her b
14 Her c
nan
0.41764722707894464
here
15 Sge b
16 Cyg B b
nan
0.7322291715776864
here
18 Del b
nan
28.13043102985347
here
1RXS 1609 b
1.7
0.15398714862670643
here
1RXS J103137.1-690205  b
nan
0.3541168367202652
here
1RXS J125608.8-692652 (AB)b
nan
0.12751331071770267
here
1RXS J131752.0-505845 b
nan
0.23630735140871564
here
1RXS J235133.3+312720 b
nan
0.02910609883423816
here
1SWASP J1407 b
nan
0.41764722707894464
here
20 LMi Ab
nan
0.8051804996227069
here
23 Lib b
nan
0.9676214028183802
here
23 Lib c
nan
0.9676214028183802
here
24 Boo b
nan
0.6644734161441683
here
24 LMi b
0.2221
2.0201780242762695
here
24 Sex b
nan
5.199834742106591
here
24 Sex c
nan


5.199834742106591
here
29 Cyg b
nan
11.469648008042189
here
2M0045+16
2M 0103-55 (AB) b
nan
0.022614429209997487
here
2M0117-34
2M 0219-39 b
1.44
0.0012662392748684374
here
2M0355
2M0418+21
2M0437 b
nan
2.399348544721082e-07
here
2M 0441+23 b
nan
6.819581837694185e-07
here
2M0444
2M0838+15 b
2M0838+15 c
2M1059-21 b
nan
0.2507643337128861
here
2M1207 A
2M1207 b
2M 1252+2735 b
2M 1253+2734 b
2M 1258+2630 b
2M1510 A
2M1510 Ab
2M1510 Ac
2M1510 B
2M 2140+16 b
0.92
0.0004685539314397822
here
2M 2206-20 b
1.3
0.0020004963936895107
here
2M 2236+4751 b
nan
0.06790787733093304
here
2MASS 0359+2009 b
nan
0.0001589777443101573
here
2MASS 1534-29 b
2MASS J0001+1535
2MASS J00103250+1715490 b
2MASS J0021-4514 b
nan
0.004986861779870669
here
2MASS J0030-1450
2MASS J0031+5749
2MASS J00413538-5621127 A
2MASS J00413538-5621127 B
2MASS J01225093-2439505 b
nan
0.022614429209997487
here
2MASS J0141-4633
2MASS J0153-6744
2MASS J0226-5327
2MASS J0249-0557A
2MASS J0249-0557 (AB)b
2MASS J0249-0557B
2MASS J0323-

2.540931069028091e-05
here
2MASS J15344984-2952274 A
2MASS J15344984-2952274 B
2MASS J1551+0941
2MASS J1554-3154 b
nan
0.12751331071770267
here
2MASS J155947+440359 b
2MASS J1617-3828 b
nan
0.07689792369877932
here
2MASS J1647+5632
2MASS J18071593+5015316
2MASS J1935-2846
2MASS J2002-0521
2MASS J2013-2806
2MASS J2117-2940
2MASS J2139+02
2MASS J2206+3301
2MASS J2206-4217
2MASS J2250+2325 b
nan
0.026698993582137367
here
2MASS J2322-6151
2MASS J2343-3646
2MASS J2352-1100
2MASSW J0829+2655
2MASSW J0928-1603
2MASSW J1326-2729
2MASSW J2208136+292121
2M J2126-81 b
nan
0.022614429209997487
here
2S 0918-549 b
0.31
3.397264880933703
here
38 Vir b
nan
1.5422851982775383
here
3 Cnc b
nan
73.99654363863702
here
41 Com b
nan
4.492433410125526
here
44 Uma b
nan
23.068320300012473
here
47 Uma b
nan
0.8051804996227069
here
47 Uma c
nan
0.8051804996227069
here
47 Uma d
nan
0.8051804996227069
here
4FGL J1408.6-2917 b
nan
3.397264880933703
here
4U 1543-62 b
0.3
3.397264880933703
here
4U 1626-67 b
0.778
3.

0.41764722707894464
here
55 Cnc Ad
nan
0.41764722707894464
here
55 Cnc Ae
0.1737
0.41764722707894464
here
55 Cnc Af
nan
0.41764722707894464
here
59 Dra b
nan
5.9836145591905625
here
59 Vir b
0.96
1.8037206127689798
here
61 Vir b
nan
0.543613382537878
here
61 Vir c
nan
0.543613382537878
here
61 Vir d
nan
0.543613382537878
here
6 Lyn b
nan
7.9993737238358325
here
70 Vir b
nan
0.4648833222492905
here
75 Cet b
nan
38.63708969707649
here
75 Cet c
nan
38.63708969707649
here
78 Psc b
nan
3.364468927189965
here
79 Cet Ab
nan
0.7322291715776864
here
81 Cet b
nan
33.36839105799381
here
82 Eri b
nan
0.2552461899192229
here
82 Eri c
nan
0.2552461899192229
here
82 Eri d
nan
0.2552461899192229
here
82 Eri e
nan
0.2552461899192229
here
83 Leo Bb
nan
0.2819484543824251
here
83 Leo Bc
nan
0.2819484543824251
here
8 Umi b
nan
10.213705168807667
here
94 Cet Ab
nan


2.783842924723662
here
AB Aur b
nan
33.36839105799381
here
AB Dor C b
nan
0.0003224637561652346
here
AB Pic b
AD 3116 b
AE 1 b
nan
2.5093397304976413
here
AF Lep b
1.33
1.6692290364820581
here
AL Com b
nan
0.06790787733093304
here
Aldebaran b
alf Ari b
nan
4.628020340357496
here
AS 205 A b
nan
0.41764722707894464
here
ASASSN-16kr b
1.05
0.5492136384792724
here
ASASSN-17jf b
1.12
0.10477441001198837
here
AU Mic b
0.4571
0.037874174113736565
here
AU Mic c
0.28
0.037874174113736565
here
AU Mic d
b Cen b
nan
743.3088986397029
here
BD-00 4475 b
BD+03 2562 b
nan
1.3100136669099762
here
BD-06 1339 c
nan
0.12751331071770267
here
BD-06 1339 d
nan
0.12751331071770267
here
BD-06 1339 e
nan
0.12751331071770267
here
BD-08 2823 b
nan
0.163893058753532
here
BD-08 2823 c
nan
0.163893058753532
here
BD-10 3166 b
1.03
0.6644734161441683
here
BD-11 4672 b
nan
0.05690938421915371
here
BD-11 4672 c
nan
0.05690938421915371
here
BD+14 4559 b
nan
0.33478453158001137
here
BD+15 2375 b
nan
1.0118526816066928
her

...

0.00012847891059136052
here
USco J155150.2-213457
UZ For (AB) b
nan
0.2987266029613343
here
UZ For (AB) c
nan
0.2987266029613343
here
V396 Hya b
VHS 1256-1257 b
1.25
0.0003391129916075793
here
VHS J183135.58-551355.9
V* V2384 Ori a
V* V2384 Ori b
W0707+24
WASP-100 b
1.69
5.6606464878063205
here
WASP-101 b
1.41
2.783842924723662
here
WASP-102 b
1.259
1.463697207053292
here
WASP-103 b
1.528
1.8037206127689798
here
WASP-104 b
1.137
0.7680406247883835
here
WASP-105 b
0.96
0.39552122222689295
here
WASP-106 b
1.085
2.174678409289415
here
WASP-107 b
0.94
0.11969570478651222
here
WASP-107 c
WASP-108 b
1.215
1.463697207053292
here
WASP-109 b
1.443
1.6889152888186156
here
WASP-10 b
1.08
0.13581973982577358
here
WASP-110 b
1.238
0.39986889429906813
here
WASP-111 b
1.442
4.628020340357496
here
WASP-112 b
1.191
0.2463498191342503
here
WASP-113 b
1.409
2.5805230414468285
here
WASP-114 b
1.339
2.329290076846474
here
WASP-117 b
1.021
1.2352314217689415
here
WASP-118 b
1.44
2.5985458801990893
here
WASP

1.6823342819527407
here
WASP-21 b
1.21
0.7322291715776864
here
WASP-22 b
1.199
1.1048736715071679
here
WASP-23 b
0.962
0.20953803373721525
here
WASP-24 b
1.3
1.2509807449936858
here
WASP-25 b
1.26
0.6977165691451518
here
WASP-26 b
1.281
1.204177863269905
here
WASP-28 b
1.213
0.7716943439276572
here
WASP-29 b
0.775
0.2738607150860144
here
WASP-2 A b
1.079
0.2987266029613343
here
WASP-30 b
0.889
1.4577775428112005
here
WASP-31 b
1.537
1.42263179296831
here
WASP-32 b
1.18
1.1048736715071679
here
WASP-33 b
1.603
4.5598652740441095
here
WASP-34 b
1.22
0.7322291715776864
here
WASP-35 b
WASP-36 b
1.269
0.7680406247883835
here
WASP-37 b
1.16
0.3145317082610643
here
WASP-38 b
1.079
1.7762060566026263
here
WASP-39 b
1.27
0.49004863498081835
here
WASP-3 b
1.454
1.9460193423166483
here
WASP-41 b
1.18
0.543613382537878
here
WASP-41 c
nan
0.543613382537878
here
WASP-42 b
WASP-43 b
1.036
0.14193681907200562
here
WASP-44 b
1.14
0.5464078708793786
here
WASP-45 b
1.16
0.4384024740363878
here
WASP-46 b
1

1.1339939352706339
here
WASP-71 b
1.5
5.692368907866253
here
WASP-72 b
1.27
3.2461714589934267
here
WASP-73 b
1.16
2.783842924723662
here
WASP-74 b
1.248
4.359721417046068
here
WASP-75 b
1.27
1.3100136669099762
here
WASP-76 b
1.83
4.1027796416331865
here
WASP-77 A b
1.21
0.7045165952994408
here
WASP-78 b
1.7
16.559864450458143
here
WASP-79 b
1.7
5.50392704760567
here
WASP-7 b
1.33
2.222667048697466
here
WASP-80 b
0.952
0.06008522101058482
here
WASP-81 b
1.429
1.0118526816066928
here
WASP-81 c
nan
1.0118526816066928
here
WASP-82 b
1.67
6.668696578284765
here
WASP-83 b
1.04
1.153724896118986
here
WASP-84 b
0.9564
0.30218041930233763
here
WASP-84 c
0.174
0.30218041930233763
here
WASP-85 A b
1.24
1.0575932104958992
here
WASP-87 A b
1.385
1.6955152615393727
here
WASP-88 b
1.7
3.978478830856325
here
WASP-89 b
1.04
0.4648833222492905
here
WASP-8 b
1.038
0.8165859457209425
here
WASP-8 c
nan
0.8165859457209425
here
WASP-90 b
1.63
5.350335272269614
here
WASP-91 b
1.03
0.2987266029613343
here
WAS

0.04740657901920443
here
WD J0914+1914 b
WD J2122+6600 b
nan
0.08187531533682886
here
Wendelstein-1 b
1.0
0.09288810277261313
here
Wendelstein-2 b
1.0
0.15398714862670643
here
WISE0047
WISE 0359-54
WISE 0855-0714
WISE 1118+3125
WISE 1711+3500 b
nan
1.1275396657766757e-05
here
WISE 2005+5424
WISE 2150A b
nan
0.0003224637561652346
here
WISEA J083011.95+283716.0
WISEA J0919-5748 b
nan
0.009095760240728515
here
WISEA J2318+0521 b
nan
0.00669844976731932
here
WISEA J2354-0811
WISE J0323+5625
WISE J0336-0143 b
nan
9.785261518629286e-08
here
WISE J041358.14-475039.3
WISE J0458+6434 A
WISE J0458+6434 B
WISE J0528+0901
WISE J0647-6232
WISE J0720-0846
nan


0.0004888689575557249
here
WISE J1217+1626
WISE J1217+1626 b
WISE J1355-8258 b
WISE J1541-2250
WISE J1617+1807
WISE J1639-6847
WISE J1741-4642
WISE J1810-1010
WISE J1812+2721
WISE J1828+2650
WISE J1828+2650 b
WISE J2018-7423
WISE J2216+1952
WISE J2313-8037
WISE J2319+7645
WISE J2359-7335
Wolf 1061 b
nan
0.009095760240728515
here
Wolf 1061 c
nan
0.009095760240728515
here
Wolf 1061 d
nan
0.009095760240728515
here
Wolf 1069 b
nan
0.0037415427537282487
here
Wolf 327 b
Wolf 359 c
Wolf 503 b
0.18227
0.11818893848499515
here
Wolf 940 b
0.92
0.010613586479217535
here
WT 5 (AB)b
nan
0.019401193724569055
here
WTS-1 b
1.49
1.6692290364820581
here
WTS-2 b
1.3
0.26596957681928984
here
XO-1 b
1.184
0.6977165691451518
here
XO-2N b
0.973
0.6324704838357619
here
XO-2 S b
nan
0.6324704838357619
here
XO-2 S c
nan
0.6387732497393187
here
XO-3 b
1.217
3.5082525645577363
here
XO-4 b
1.317
2.5985458801990893
here
XO-5 b
1.03
0.37435345025165656
here
XO-6 b
2.07
4.22984868381296
here
XO-7 b
1.346
3.4524368257

In [7]:
radius_data = np.array(radius_data) ; mass_data = np.array(mass_data)
radius_estimate = np.array(radius_estimate)
radius_estimate_expansion = np.array(radius_estimate_expansion)
star_distance = np.array(star_distance)
temp_eq = np.array(temp_eq)

In [8]:
mask_plot = (~np.isnan(radius_data)) #& (star_distance <= 2.)# & (mass_data <= 1.)#& (radius_estimate_expansion <= 10.)

#print("L'expansion moyenne est de :",np.nanmean(radius_estimate_expansion[mask_plot]-radius_estimate[mask_plot]), " RJ.")

In [ ]:
data_model = pd.DataFrame({'radius_m' : radius_data[mask_plot]*RJ, 'radius_pred_m' : radius_estimate[mask_plot]*RJ,'mass_kg' : mass_data[mask_plot]*MJ, 'temp_eq_K':temp_eq[mask_plot]})

data_model.to_csv('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/radius_expansion.csv', index = False)

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(np.sort(radius_data[mask_plot]),radius_data[mask_plot][np.argsort(radius_data[mask_plot])], linestyle='dashed', color='tab:blue', label = 'Measured values')
plt.scatter(radius_estimate[mask_plot],radius_data[mask_plot], marker ='x', color = 'tab:orange', label = 'Estimated values')
plt.scatter(radius_estimate_expansion[mask_plot],radius_data[mask_plot], marker = 'x', color='tab:green', label='Estimated values with expansion')
plt.xlabel('R_pred [RJ]')
plt.ylabel('R_measured [RJ]')
plt.title('Evolution of measured radius values compared to predicted values.')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/fit_mass_radius_with_expT.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

## Correlation between predicted and measured values

In [9]:
def correlation(a,b):
    A = np.nanmean(a)
    B = np.nanmean(b)
    AB = np.nanmean(a*b)

    return (AB - A*B)/(np.nanstd(a)*np.nanstd(b))

In [ ]:
correl_est = correlation(radius_data[mask_plot],radius_estimate[mask_plot])
correl_exp = correlation(radius_data[mask_plot],radius_estimate_expansion[mask_plot])

print(correl_est )
print(correl_exp )
print(np.std(radius_data[mask_plot]))
print(np.std(radius_estimate[mask_plot]))
print(np.std(radius_estimate_expansion[mask_plot]))

In [ ]:
print(np.min(radius_estimate[mask_plot]),np.max(radius_estimate[mask_plot]))
print(np.min(radius_estimate_expansion[mask_plot]),np.max(radius_estimate_expansion[mask_plot]))

In [ ]:
print(np.correlate(np.sort(radius_data[mask_plot]),np.sort(radius_data[mask_plot])))
print(chi2(radius_data[mask_plot],radius_data[mask_plot]).value)
print(correl_est)

In [ ]:
plt.plot(radius_data[mask_plot]/radius_data[mask_plot],radius_estimate[mask_plot]/radius_data[mask_plot])
plt.plot(radius_data[mask_plot]/radius_data[mask_plot],radius_estimate_expansion[mask_plot]/radius_data[mask_plot])
plt.grid()
plt.show()

In [ ]:
print(radius_estimate_expansion[mask_plot]/radius_data[mask_plot])

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(np.sort(radius_data[mask_plot]),radius_data[mask_plot][np.argsort(radius_data[mask_plot])], linestyle='dashed', color='tab:blue', label = 'Data')
plt.scatter(radius_data[mask_plot],radius_estimate[mask_plot], marker ='o', color = 'tab:orange', label = 'Estimated values\n correl = {:2f}'.format(correl_est))
plt.scatter(radius_data[mask_plot],radius_estimate_expansion[mask_plot], marker = 'o', color='tab:green', label='Estimated values with expansion \n correl = {:2f}'.format(correl_exp))
plt.xlabel('R$_{measured}$ [RJ]')
plt.ylabel('Planetary radius [RJ]')
plt.title('Evolution of estimated radius values compared to known values.')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/fit_mass_radius_with_expT.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

## Radius as a function of mass :

In [ ]:
mass = np.logspace(-2,1.4,500)
R = np.array([Planet._calculate_radius(models = config_param.planet_radius_models, mass = mass[i]) for i in range(500)])


In [ ]:
corr_pred = correlation(radius_data[mask_plot],radius_estimate[mask_plot])
corr_est = correlation(radius_data[mask_plot],radius_estimate_expansion[mask_plot])

In [ ]:
print(corr_est,corr_pred)

In [ ]:
plt.figure(figsize=(10,7))
#plt.plot(mass,R, linestyle = 'dashed',color = 'black',label='model')
plt.scatter(mass_data[mask_plot],radius_data[mask_plot], marker='+', color='tab:blue', label = 'Mesures')
plt.scatter(mass_data[mask_plot],radius_estimate[mask_plot], marker ='+', color = 'tab:orange', label = '$R_{p,pred}$')#\n correl = {:2f}'.format(correl_est))
plt.scatter(mass_data[mask_plot],radius_estimate_expansion[mask_plot], marker = '+', color='tab:green', label="$R_{est}$")#\n correl = {:2f}'.format(correl_exp))
plt.xlabel('$M_p$ [$M_J$]',fontsize=14)
plt.ylabel('$R_p$ [$R_J$]',fontsize=14)
#plt.title('Evolution of radius values compared to mass.')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/Rp_vs_Mp.png', transparent=True, bbox_inches='tight', dpi=150)
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/ch3/fig/plot_Rp_vs_Mp.png', dpi=150 )
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
#plt.plot(,R, linestyle = 'dashed',color = 'black',label='model')
plt.scatter(star_distance[mask_plot],radius_data[mask_plot], marker='o', color='tab:blue', label = 'Data')
plt.scatter(star_distance[mask_plot],radius_estimate[mask_plot], marker ='^', color = 'tab:orange', label = 'Estimated values')
plt.scatter(star_distance[mask_plot],radius_estimate_expansion[mask_plot], marker = '+', color='tab:green', label='Estimated values with expansion\n correl = {:2f}'.format(correl_exp))
plt.xlabel('Star distance [UA]')
plt.ylabel('Planetary radius [RJ]')
plt.title('Evolution of radius values compared to mass.')
plt.xscale('log')
plt.legend()
plt.grid()
plt.show()

## Radius as a function of temperature and semi-major axis

In [ ]:
fig,ax = plt.subplots(figsize=(10,7))
ax.scatter(star_distance[mask_plot],temp_eq[mask_plot], c=radius_data[mask_plot], cmap = 'viridis')
plt.xscale('log')
plt.xlabel('Star-planet distance [AU]')
plt.ylabel('Equilibrium temperature [K]')
plt.colorbar(label='Planetary Radius [RJ]')
plt.grid()
plt.show()

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(4 * 3, 4 * 3))
#.title('$\Delta$RM as a function of RM, during day/night time')

sc1 = ax[0,0].scatter(star_distance[mask_plot],temp_eq[mask_plot], c=radius_data[mask_plot], cmap = 'viridis')
ax[0,0].set_xlabel('Star-planet distance [AU]')
ax[0,0].set_ylabel('Equilibrium temperature [K]')
ax[0,0].set_xscale('log')
cax1 = inset_axes(
            ax[0,0],
            width='3%',
            height='100%',
            loc='lower left',
            bbox_to_anchor=(1.03, 0., 1, 1),
            bbox_transform=ax[0,0].transAxes,
            borderpad=0,
        )

cbar1 = plt.colorbar(sc1, cax=cax1)
cbar1.set_label('Planetary Radius [RJ]')
ax[0,0].grid()

sc2 = ax[0,1].scatter(star_distance[mask_plot],radius_data[mask_plot], c=temp_eq[mask_plot], cmap = 'viridis')
ax[0,1].set_xlabel('Star-planet distance [AU]')
ax[0,1].set_ylabel('Planetary Radius [RJ]')
ax[0,1].set_xscale('log')
cax2 = inset_axes(
            ax[0,0],
            width='3%',
            height='100%',
            loc='lower left',
            bbox_to_anchor=(1.03, 0., 1, 1),
            bbox_transform=ax[0,1].transAxes,
            borderpad=0,
        )

cbar2 = plt.colorbar(sc2, cax=cax2)
cbar2.set_label('Equilibrium temperature [K]')
ax[0,1].grid()


sc3 = ax[1,0].scatter(temp_eq[mask_plot],radius_data[mask_plot], c=np.log10(star_distance[mask_plot]), cmap = 'viridis')
ax[1,0].set_xlabel('Equilibrium temperature [K]')
ax[1,0].set_ylabel('Planetary Radius [RJ]')
ax[1,0].set_xscale('log')
cax3 = inset_axes(
            ax[0,0],
            width='3%',
            height='100%',
            loc='lower left',
            bbox_to_anchor=(1.03, 0., 1, 1),
            bbox_transform=ax[1,0].transAxes,
            borderpad=0,
        )

cbar3 = plt.colorbar(sc3, cax=cax3)
cbar3.set_label('Star-planet distance [AU]')
ax[1,0].grid()


fig.subplots_adjust(wspace=0.4, hspace=0.2)

## Updated version for radius expansion

In [10]:
def poly_func(coeffs, x) :
    res = 0
    for i,c in enumerate(coeffs[::-1]):
        res +=  c*np.power(x,i)
    return res

### 1 : No radius expansion : new mass-radius relation with polyfit

In [ ]:
mass_sorted = np.sort(mass_data[mask_plot]) ; radius_sorted = radius_data[mask_plot][np.argsort(mass_data[mask_plot])]
plt.figure(figsize=(10,7))
plt.plot(mass_sorted,radius_sorted, marker='x', color='tab:blue', label = 'Data')
for i in range(1,12):
    coeffs = np.polyfit(np.log10(mass_sorted),np.log10(radius_sorted), deg = i)
    if i==9 :
        print(coeffs[::-1])
        plt.plot(mass_sorted,10**poly_func(coeffs,np.log10(mass_sorted)), linestyle='dashed',color='tab:red',label='ordre'+str(i))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Planetary mass [MJ]')
plt.ylabel('Planetary radius [RJ]')
plt.title('Evolution of radius with respect to mass.')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/fit_mass_radius.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

- Test new method : polyfit order 9

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(mass_data[mask_plot],radius_data[mask_plot], marker='o', color='tab:blue', label = 'Data')
plt.scatter(mass_data[mask_plot],radius_estimate[mask_plot], marker ='o', color = 'tab:orange', label = 'Estimated values')
plt.xlabel('Planetary mass [MJ]')
plt.ylabel('Planetary radius [RJ]')
plt.title('Evolution of radius values compared to mass.')
plt.xscale('log')
#plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/polyfit_mass_radius_loglin.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

### 2 : Same mass-radius relation as before, but new model for radius expansion

In [11]:
expansion_factor = radius_data[mask_plot]/radius_estimate[mask_plot]

mass_sorted = np.sort(mass_data[mask_plot]) ; expansion_sorted = expansion_factor[np.argsort(mass_data[mask_plot])]
radius_sorted = radius_data[mask_plot][np.argsort(mass_data[mask_plot])]
#Teq_sorted = expansion_factor[np.argsort(temp_eq[mask_plot])]

plt.figure(figsize=(10,7))
plt.scatter(mass_sorted,expansion_sorted, marker='o', color='tab:blue', label = 'Data')
for i in range(1,15):
    coeffs = np.polyfit(np.log10(mass_sorted),np.log10(expansion_sorted), deg = i)
    correl, _ = pearsonr(expansion_sorted, 10**poly_func(coeffs,np.log10(mass_sorted)))
    if i==9 :
        print(coeffs[::-1])
        plt.plot(mass_sorted,10**poly_func(coeffs,np.log10(mass_sorted)), color='tab:red',linestyle='dashed',label='Ordre {}, c={:1.3f}'.format(i,correl))

plt.xlabel('Planetary mass [MJ]')
plt.ylabel('Radius expansion [dimensionless]')
plt.title('Evolution of radius expansion with respect to mass.')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/expansion_vs_mass.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(star_distance[mask_plot],expansion_factor, marker='o', color='tab:blue', label = 'Data')
plt.xlabel('Star-planet distance [UA]')
plt.ylabel('Radius expansion [dimensionless]')
plt.title('Evolution of radius expansion with respect to star-planet distance.')
plt.xlim(1e-3,10)
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/expansion_vs_stardist.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

- test polyfit of order 9 :

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(mass_sorted,radius_sorted, marker='o', color='tab:blue', label = 'Data')
plt.scatter(mass_data[mask_plot],radius_estimate_expansion[mask_plot], marker = 'o', color='tab:red', label='Estimated with expansion')
plt.xlabel('Planetary mass [MJ]')
plt.ylabel('Radius [RJ]]')
plt.title('Evolution of radius with expansion with respect to mass.')
plt.xscale('log')
#plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/radius_exp_vs_mass_loglin.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

### Fit de Expansion(Teq)

In [12]:
#expansion_factor = radius_data[mask_plot]/radius_estimate[mask_plot]
mask_fit = (expansion_factor > 0.8) & (expansion_factor < 1.9) & (temp_eq[mask_plot] > 400)

expansion_sorted = np.sort(expansion_factor[mask_fit])
Teq_sorted = temp_eq[np.argsort(expansion_factor[mask_fit])]

#coeffs = np.polyfit(np.log10(Teq_sorted),expansion_sorted, deg = 2)
coeffs = np.polyfit(np.log10(temp_eq[mask_plot][mask_fit]),expansion_factor[mask_fit], deg = 2)
correl, _ = pearsonr(expansion_factor[mask_fit], poly_func(coeffs,np.log10(temp_eq[mask_plot][mask_fit])))

In [ ]:
print(coeffs[::-1])


In [ ]:
coeffs_teq = [0.93174824 ,-4.78521151 , 7.13173118]

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(temp_eq[mask_plot],expansion_factor, marker='o', color='tab:blue', label = 'Mesures')
#plt.scatter(Teq_sorted,expansion_sorted, color='red')
plt.scatter(temp_eq[mask_plot][mask_fit], poly_func(coeffs_teq,np.log10(temp_eq[mask_plot][mask_fit])), marker = 'x', color='tab:red', label='Fit polynomial')
plt.hlines(1.9, xmin = 0.2,xmax =6000,linestyles='dashed', color='black')
plt.hlines(0.8, xmin = 0.2,xmax =6000,linestyles='dashed', color='black')
plt.vlines(400, ymin=0,ymax=6,linestyles='dashed', color='black')
plt.xlabel('$T_{eq}$ [K]')
plt.ylabel('$\eta_{exp}$')
#plt.title('Evolution of expansion with respect to temperature.')
plt.xlim(40,6000)
plt.ylim(0,3)
plt.xscale('log')
#plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/exp_vs_Teq.png', transparent=True, bbox_inches='tight', dpi=150)
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/an2/fig/plot_exp_vsTeq.png', dpi=150 )
plt.show()

## Fit de R_meas(R_pred * exp)

In [14]:
radius_estimate_expansion_old = np.copy(radius_estimate_expansion)

In [15]:
mask_fit_2 = (~np.isnan(radius_data)) & (~np.isnan(radius_estimate_expansion_old))
coeffs_2 = np.polyfit(np.log10(radius_estimate_expansion_old[mask_fit_2]),np.log10(radius_data[mask_fit_2]), deg = 4)
correl_2, _ = pearsonr(radius_data[mask_fit_2], 10**poly_func(coeffs_2,np.log10(radius_estimate_expansion_old[mask_fit_2])))
print(coeffs_2[::-1])

[-0.03754891  1.50464295 -1.98539555 -4.74575712 -2.49993389]


In [16]:
coeffs_lin = [0.655981 , -0.90527977, -1.19830686, 1.39140901, -0.06070923 ]

In [18]:
plt.figure(figsize=(10,7))
plt.scatter(radius_estimate_expansion[mask_fit_2],radius_data[mask_fit_2], marker='o', color='tab:blue', label = '$R_{p,pred}$ x $\eta_{exp}$')
#plt.scatter(radius_estimate_expansion[mask_fit_2],radius_estimate_expansion[mask_fit_2]*(10**poly_func(coeffs_2,np.log10(radius_estimate_expansion[mask_fit_2]))), marker='+', color='tab:orange')
plt.scatter(radius_estimate_expansion[mask_fit_2],10**poly_func(coeffs_lin,np.log10(radius_estimate_expansion[mask_fit_2])), marker='x', color='tab:red',label='Fit, order 4')
plt.plot([0.08,2.5],[0.08,2.5], linestyle='dashed', color='black', label='Mesures')
plt.xlabel('$R_{p,pred}$ x $\eta_{exp}$ [RJ]',fontsize=14)
plt.ylabel('$R_{mes}$ [RJ]',fontsize=14)
#plt.title('Evolution of radius measured with respected to predicted radius with expansion')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/Rpred_vs_Rmeas.png', transparent=True, bbox_inches='tight', dpi=150)
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/an2/fig/plot_Rmes_vs_Rexp.png', dpi=150 )
plt.show()

In [20]:
plt.figure(figsize=(10,7))
#plt.scatter(radius_estimate_expansion_old[mask_fit_2],radius_data[mask_fit_2], marker='x', color='tab:blue', label = '$R_{p,pred}$ x $\eta_{exp}$')
plt.scatter(10**poly_func(coeffs_2,np.log10(radius_estimate_expansion_old[mask_fit_2])),radius_data[mask_fit_2], marker='x', color='tab:red',label='$R_{est}$')
plt.scatter(radius_estimate[mask_fit_2],radius_data[mask_fit_2], marker='o', color='tab:blue', label='$R_{p,pred}$')
plt.plot([0.08,2.5],[0.08,2.5], linestyle='dashed', color='black',label='Mesures')
plt.xlabel('$R$ [RJ]',fontsize=14)
plt.ylabel('$R_{mes}$ [RJ]',fontsize=14)
#plt.title('Evolution of radius measured with respected to predicted radius with expansion')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/Rpred_vs_Rmeas.png', transparent=True, bbox_inches='tight', dpi=150)
plt.savefig('/Users/emauduit/Documents/These/phd-manuscript/an2/fig/plot_Rmes_vs_Rexpcorr.png', dpi=150 )
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(radius_estimate_expansion[mask_fit_2],10**poly_func(coeffs_2,np.log10(radius_estimate_expansion[mask_fit_2]))/radius_estimate_expansion[mask_fit_2], marker='o', color='tab:blue', label = 'Data')
#plt.plot([0.08,2.5],[0.08,2.5], linestyle='dashed', color='black')
plt.xlabel('Rp * E')
plt.ylabel('Rp* E corrigé')
plt.title('Evolution of radius measured with respected to predicted radius with expansion')
plt.xscale('log')
plt.yscale('log')
plt.legend()
plt.grid()
#plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/Rpred_vs_Rmeas.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()

## Recomputing constants for the M_p-R_p relation from Griessmeier, 2017, A&A

In [ ]:
def mass_radius_relation(Mp,rho0,Mmax) :
    RJ = 69911e3
    res = np.power((4 / 3) * np.pi * rho0, -1.0 / 3) * np.power(Mp * 1.8986e27, 1.0 / 3) / (1 + np.power(Mp / Mmax, 2.0 / 3))
    return res / RJ

In [ ]:
mask_plot = (~np.isnan(radius_data)) #& (mass_data >= 0.01) & (mass_data <= 10.)
mass_sorted = np.sort(mass_data[mask_plot])
radius_sorted = radius_data[mask_plot][np.argsort(mass_data[mask_plot])]

In [ ]:
from scipy.optimize import curve_fit

popt, pcov = curve_fit(mass_radius_relation,mass_sorted,radius_sorted, p0=[394,3.16])

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(mass_sorted,radius_sorted, marker='o', color='tab:blue', label = 'Data')
plt.scatter(mass_sorted,mass_radius_relation(mass_sorted,*popt), marker = 'x', color='tab:red', label='New params, p_0 = {:3f}, M_max = {:3f}'.format(*popt))
plt.scatter(mass_sorted,mass_radius_relation(mass_sorted,394,3.16), marker = 'x', color='tab:orange', label='Old params, p_0 = {394 kg/m3}, M_max = {3.16 MJ}')
plt.xlabel('Planetary mass [MJ]')
plt.ylabel('Radius [RJ]]')
plt.title('Evolution of radius with expansion with respect to mass.')
plt.xscale('log')
#plt.yscale('log')
plt.legend()
plt.grid()
plt.savefig('/Users/emauduit/Documents/These/target_selection/Etudes/Planetary radius expansion/new_fit.png', transparent=True, bbox_inches='tight', dpi=150)
plt.show()